## **1. The corpus**

In [ ]:
# !wget --no-check-certificate \
#     https://gist.githubusercontent.com/khacanh/4c4662fa226db87a4664dfc2f70bc63e/raw/5d8a1d890c73a1e92e6898137db28f3dc0676975/kieu.txt \
#     -O ./kieu.txt

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
text = []
f = open("/content/drive/MyDrive/word-embedding-creation/input/text8_short", "r")
for line in f:
  text.append(line)

In [3]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence

In [4]:
corpus = []
for i in range(len(text)):
  corpus.append(text_to_word_sequence(text[i]))

In [5]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(corpus)
w2id = tokenizer.word_index

## **2. Preprocess data for Skip-gram**

In [6]:
vocab_size = len(tokenizer.word_index) + 1
window_size = 2

In [7]:
import numpy as np
def generate_pairs(window_size, corpus):

  X = []
  y = []
  for sent in corpus:
    tar_i = 0
    while tar_i < len(sent):
      start = max(0, tar_i-2)
      end = min(len(sent)-1, tar_i+2)

      labels = sent[start:tar_i] + sent[tar_i+1:end+1]
      x = [' '.join([sent[tar_i]] * len(labels))]
      labels = [' '.join(labels)]
      # print(sent)
    #   print(x, "--->", labels)
      tar_i += 1
    #   print(tokenizer.texts_to_sequences(x)[0])
    #   print(tokenizer.texts_to_sequences(labels)[0])
      X.extend(tokenizer.texts_to_sequences(x)[0])
      y.extend(tokenizer.texts_to_sequences(labels)[0])
      

  return tf.convert_to_tensor(X) , tf.convert_to_tensor(y)


In [8]:
X_train, y_train = generate_pairs(window_size, corpus)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda
from tensorflow.keras.backend import mean 

In [10]:
X_train = tf.expand_dims(X_train, axis=1)
y_train = tf.expand_dims(y_train, axis=1)

In [11]:
print(X_train.shape) 
print(y_train.shape)

(892978, 1)
(892978, 1)


In [12]:
vocab_size

20376

## **3. Skip_gram**

In [ ]:
top4_accuracy_metric = tf.metrics.SparseTopKCategoricalAccuracy(k=4, name='top4_acc')
embedding_size = 300
skip_gram = Sequential()
skip_gram.add(Embedding(input_dim=vocab_size, output_dim=embedding_size))
skip_gram.add(Dense(vocab_size, activation='softmax'))

skip_gram.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=[top4_accuracy_metric])
skip_gram.fit(X_train, y_train, epochs=30, verbose=1)

Epoch 1/30
27906/27906 [==============================] - 364s 13ms/step - loss: 7.2242 - top4_acc: 0.1934
Epoch 2/30
 1614/27906 [>.............................] - ETA: 5:43 - loss: 6.5645 - top4_acc: 0.2201

## **4. Some intuition 1**

### *4.1. top4 accuracy*

Thay vì sử dụng metric accuracy như notebook CBOW, em sử dụng TopKAccuracy vì mỗi một input vào có thể có `2*window_size` outputs, vì vậy nếu target của một sample chỉ cần nằm trong top `2*window_size` có softmax score cao nhất thì đã có thể đánh giá model đã perform tốt trên sample đó. 

### *4.2. Dense layer as another Embedding:*

Số lượng parameters của lớp `Dense` trong model được huấn luyện phía trên là `embedding_dim * vocab_size`. Có thể nói số lượng weights trong lớp `Embedding` và lớp `Dense` là giống nhau. Tuy nhiên, cách hoạt động ngược nhau: Lớp `Embedding` nhận vào một `one_hot vector` và cho ra embedding của từ tương ứng với `one_hot` đó; còn Dense nhận vào embedding và được train để cho ra `one_hot vector` (tất nhiên nó sẽ không thể cho ra một `one_hot vector` :">)

### *4.3. The inefficiency of learning process:*

Từ cross_entropy_loss function: $L=−y⋅log(y_{predicted}\hspace{0.001em})$

Ta có thể thấy, learning process chỉ tập trung vào target word (từ có giá trị tương ứng trong target one_hot vector bằng 1) và hoàn toàn bỏ qua tất cả những embedding của negative words. Điều này dường như rất bất hợp lí, bởi vì để compute được logits vector để đưa vào cross_entropy_loss thì cũng phải tính toán phép nhân một in_embedding_vector với out_embedding_vector**s** của toàn bộ vocabulary. Hay nói cách khác, nếu chỉ đơn thuần train với loss function là cross_entropy_loss thì mỗi lần backpropagation, chúng ta chỉ update ít hơn 1% (1/118) weights của lớp Dense. It's totally ineffecient.

## **5. Skip Gram with negative sampling**

Negative sampling formula:

$$P(w _{i}\hspace{0.001em}) = \frac{f(w _{i}\hspace{0.001em})^{3/4}}{\sum_0^n (f(w _{j}\hspace{0.001em})^{3/4})}$$

$P(w _{i}\hspace{0.001em})$: xác suất từ $w _{i}\hspace{0.001em}$ sẽ được chọn làm negative words

$f(w _{i}\hspace{0.001em}$: số lần $w _{i}\hspace{0.001em}$ suất hiện trong corpus

In [13]:
from collections import Counter
int_corpus = []
for i in range(len(text)):
  text_seq = text_to_word_sequence(text[i])
  for word in text_seq:
    int_corpus.append(w2id[word])

appear_counts = Counter(int_corpus)
total_count = len(int_corpus)
freqs_dict = {word: count/total_count for word, count in appear_counts.items()}

freqs_arr = np.array(sorted(freqs_dict.values(), reverse=True))
sampling = tf.convert_to_tensor(freqs_arr**(0.75)/np.sum(freqs_arr**(0.75)))
sampling = tf.expand_dims(sampling, axis=0)

In [17]:
print(sampling.shape)

(1, 20374)


In [16]:
n_samples = 10
negative_samples = tf.compat.v1.multinomial(sampling, y_train.shape[0] * n_samples)
negative_samples = tf.reshape(negative_samples, [y_train.shape[0], n_samples])
print(negative_samples[0])

ResourceExhaustedError: ignored

In [ ]:
class SkipGramNeg(tf.keras.Model):
  def __init__(self, vocab_size, embedding_size):
    super().__init__()
    
    self.vocab_size = vocab_size
    self.embedding_size = embedding_size
    
    # define embedding layers for input and output words
    self.in_embed = Embedding(vocab_size, embedding_size)
    self.out_embed = Embedding(vocab_size, embedding_size)
  def call(self, inputs, targets, negative_samples):
    input_vectors = self.in_embed(inputs)
    target_vectors = self.out_embed(targets)
    negative_vectors = self.out_embed(negative_samples)

    return input_vectors, target_vectors, negative_vectors

In [ ]:
from tensorflow import math

def negativeSamplingLoss(input_vectors, output_vectors, negative_vectors):
    input_vectors =  tf.transpose(input_vectors, perm=(0,2,1))
    out_loss = math.log(math.sigmoid(tf.matmul(output_vectors, input_vectors)))
    out_loss = tf.squeeze(out_loss)
    
    # incorrect log-sigmoid loss

    negative_loss = math.log(math.sigmoid(tf.matmul(math.negative(negative_vectors), input_vectors)))
    negative_loss = math.reduce_sum(tf.squeeze(negative_loss), axis=1)  # sum the losses over the sample of noise vectors

    # negate and sum correct and noisy log-sigmoid losses
    # return average batch loss
    return tf.math.reduce_mean(-(out_loss + negative_loss))

In [ ]:
embedding_size = 128
neg_skip_gram = SkipGramNeg(vocab_size, embedding_size)
loss_fn = negativeSamplingLoss
optimizer = tf.keras.optimizers.Adam()

In [ ]:
epochs = 300
for epoch in range(epochs):
  print("\nStart of epoch %d" % (epoch,))

  with tf.GradientTape() as tape:

      # Run the forward pass of the layer.
      # The operations that the layer applies
      # to its inputs are going to be recorded
      # on the GradientTape.
      input_vectors, output_vectors, negative_vectors = neg_skip_gram( X_train, y_train, negative_samples)  # Logits for this minibatch
      # Compute the loss value for this minibatch.
      loss_value = loss_fn(input_vectors, output_vectors, negative_vectors)

  # Use the gradient tape to automatically retrieve
  # the gradients of the trainable variables with respect to the loss.
  grads = tape.gradient(loss_value, neg_skip_gram.trainable_weights)

  # Run one step of gradient descent by updating
  # the value of the variables to minimize the loss.
  optimizer.apply_gradients(zip(grads, neg_skip_gram.trainable_weights))

  print(
      "Training loss: %.4f"
      % (float(loss_value))
  )



Start of epoch 0
Training loss: 7.6248

Start of epoch 1
Training loss: 7.6192

Start of epoch 2
Training loss: 7.6137

Start of epoch 3
Training loss: 7.6080

Start of epoch 4
Training loss: 7.6023

Start of epoch 5
Training loss: 7.5964

Start of epoch 6
Training loss: 7.5903

Start of epoch 7
Training loss: 7.5839

Start of epoch 8
Training loss: 7.5773

Start of epoch 9
Training loss: 7.5703

Start of epoch 10
Training loss: 7.5630

Start of epoch 11
Training loss: 7.5552

Start of epoch 12
Training loss: 7.5469

Start of epoch 13
Training loss: 7.5381

Start of epoch 14
Training loss: 7.5287

Start of epoch 15
Training loss: 7.5187

Start of epoch 16
Training loss: 7.5080

Start of epoch 17
Training loss: 7.4965

Start of epoch 18
Training loss: 7.4842

Start of epoch 19
Training loss: 7.4711

Start of epoch 20
Training loss: 7.4571

Start of epoch 21
Training loss: 7.4421

Start of epoch 22
Training loss: 7.4261

Start of epoch 23
Training loss: 7.4090

Start of epoch 24
Trainin

## **6. Some intuition 2**

\**Question:** Em đang chưa hiểu lắm về việc tại sao người ta thường dùng `in_embedding` để làm transferring embedding cho model khác ạ?


*Possible intutition:* Với cặp từ 'đầu', 'lòng' thì tasks của 2 Embedding layers là làm cho vector tương ứng với 'đầu' ở in_embed giống với vector của 'lòng' ở out_embed. Bởi vì như thế sẽ maximize được $(u \cdot v)$ và minimize $-log(u \cdot v)$

Bên cạnh đó, tập dataset cũng có tính đối xứng giữa inputs và labels.

Vậy có phải việc lấy `in_embed` làm `transferring embedding` cho model khác là random không ạ? Nếu với tập train không nhiều samples thì có nên dùng `out_embed` để làm `transferring embedding` không ạ? Tại vì với `negative_sampling`, thì `out_embed` mỗi một lần `backpropagation` update nhiều weights hơn so với `in_embed`.